In [1]:
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd

from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

In [30]:
df = pd.read_parquet('../data/SO_2014_2022.pq')

avg_salary = df.groupby(['YearsCodePro', 'Gender', 'Year'])['Salary'].mean().reset_index()

fig = px.line(avg_salary, x="YearsCodePro", y="Salary", animation_frame="Year", color="Gender",
              range_x=[0, 50], range_y=[0, 250000])
fig.update_layout(xaxis_title="YearsCodePro", yaxis_title="Salary")

fig.show()